<a href="https://colab.research.google.com/github/fedjkeee/NLP_-Deeplearning/blob/main/Project_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
#import tokenization
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


– id: уникальный номер сообщения в системе twitter; 

– tdate: дата публикации сообщения (твита); 
– tmane: имя пользователя, опубликовавшего сообщение; 
– ttext:  текст сообщения (твита);
– ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
– trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
– tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
– tstcount: число всех сообщений пользователя в сети twitter;
– tfol: количество фоловеров пользователя (тех людей, которые читают пользователя);
– tfrien: количество друзей пользователя (те люди, которых читает пользователь);
– listcount: 

In [3]:
columns=['id','tdate','tmane','ttext','ttype','terp','trtw','tfav','tstcount','tfol','tfrien','listcount']

In [4]:
df_p = pd.read_csv("/content/drive/MyDrive/positive.csv",sep=";",header=None)
df_n = pd.read_csv("/content/drive/MyDrive/negative.csv",sep=";",header=None)

In [5]:
df_p.columns=columns
df_n.columns=columns

In [6]:
df_p.head()

,id,tdate,tmane,ttext,ttype,terp,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [7]:
df_n.head()

,id,tdate,tmane,ttext,ttype,terp,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [8]:
df=pd.concat([df_n,df_p])

In [9]:
df['ttype'].value_counts()

 1    114911
-1    111923
Name: ttype, dtype: int64

In [10]:
df.head()

,id,tdate,tmane,ttext,ttype,terp,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [11]:
df.head()

,id,tdate,tmane,ttext,ttype,terp,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [12]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [13]:
X = df['ttext']
y = df['ttype'].apply(lambda x: 1 if x == 1 else 0)

In [14]:
X.reset_index(drop=True, inplace=True)

In [15]:
X.loc[0]

'остаются самые нужные и самые близкие) все остальные уходят) и я только рада) потому что я никогда сама не пойму нужен мне человек или нет)'

In [16]:
texts = X.values.tolist()
labels = y.values.tolist()

##### Train test val split

In [17]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=2, stratify=labels, shuffle=True)
print("train_texts: ", len(train_texts))
print("test_texts: ", len(test_texts))
print("train_labels: ", len(train_labels))
print("test_labels: ",len(test_labels))

train_texts:  181467
test_texts:  45367
train_labels:  181467
test_labels:  45367


In [18]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=2, stratify=train_labels ,shuffle=True)
print("train_texts: ", len(train_texts))
print("val_texts: ", len(val_texts))
print("train_labels: ", len(train_labels))
print("val_labels: ", len(val_labels))

train_texts:  145173
val_texts:  36294
train_labels:  145173
val_labels:  36294


#### IMBDB
https://huggingface.co/transformers/custom_datasets.html

In [19]:
!pip install transformers

In [20]:
import tensorflow as tf
import transformers
from tensorflow.keras.layers import Dense, Input, Dropout
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [21]:
model_checkpoint = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#transformer_model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [22]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [23]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

Fine-tuning with Trainer

In [24]:
from transformers import  Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results_',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=20, #10
    save_steps = 8000
)

model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence" )

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
20,0.683600
40,0.652400
60,0.586800
80,0.385000
100,0.169900
120,0.053600
140,0.046200
160,0.023500
180,0.046800
200,0.024500


Step,Training Loss
20,0.683600
40,0.652400
60,0.586800
80,0.385000
100,0.169900
120,0.053600
140,0.046200
160,0.023500
180,0.046800
200,0.024500


TrainOutput(global_step=9074, training_loss=0.011040587667204785, metrics={'train_runtime': 2505.6732, 'train_samples_per_second': 3.621, 'total_flos': 21068908428494304, 'epoch': 1.0})

In [25]:
'''from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(1):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()'''

'from torch.utils.data import DataLoader\nfrom transformers import DistilBertForSequenceClassification, AdamW\n\ndevice = torch.device(\'cuda\') if torch.cuda.is_available() else torch.device(\'cpu\')\n\nmodel = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence")\nmodel.to(device)\nmodel.train()\n\ntrain_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)\n\noptim = AdamW(model.parameters(), lr=5e-5)\n\nfor epoch in range(1):\n    for batch in train_loader:\n        optim.zero_grad()\n        input_ids = batch[\'input_ids\'].to(device)\n        attention_mask = batch[\'attention_mask\'].to(device)\n        labels = batch[\'labels\'].to(device)\n        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)\n        loss = outputs[0]\n        loss.backward()\n        optim.step()\n\nmodel.eval()'

In [26]:
#model.save_pretrained("/content/drive/MyDrive/pretrained_rubert_2_/")

In [27]:
eva = trainer.evaluate()

In [28]:
#trainer.predict(test_dataset)

In [29]:
class IMDbDataset_test(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(test_labels)

In [30]:
test_nolab = IMDbDataset_test(test_encodings)

In [31]:
preds = trainer.predict(test_nolab)

In [32]:
pred_lab =[i.argmax() for i in preds[0]]

In [33]:
from sklearn.metrics import classification_report

In [34]:
classification_report(pred_lab,test_labels)

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     22383\n           1       1.00      1.00      1.00     22984\n\n    accuracy                           1.00     45367\n   macro avg       1.00      1.00      1.00     45367\nweighted avg       1.00      1.00      1.00     45367\n'

In [47]:
preds_2 = trainer.predict(test_dataset.encodings)

ValueError: ignored

In [60]:
preds[0].apply(lambda x: argmax(x))

AttributeError: ignored

In [55]:
preds[1][1]

1

In [27]:
predictions = [i.argmax() for i in preds[0]]

In [28]:
accuracy_score(predictions,test_labels)

0.9996252782859788

In [29]:
from sklearn.metrics import classification_report

In [30]:
 classification_report(predictions,test_labels)

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     22372\n           1       1.00      1.00      1.00     22995\n\n    accuracy                           1.00     45367\n   macro avg       1.00      1.00      1.00     45367\nweighted avg       1.00      1.00      1.00     45367\n'

In [67]:
test_dataset = IMDbDataset(test_encodings, labels=None)

In [68]:
preds_nolab = trainer.predict(test_dataset)

TypeError: ignored

In [83]:
preds[0][3]

array([ 6.0237594, -4.6692243], dtype=float32)

In [26]:
model_2 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/pretrained_rubert/")

In [32]:
trainer_2 = Trainer(
    model=model_2,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [93]:
preds_mod_2 = trainer_2.predict(test_dataset)

In [94]:
predictions_mod_2 = [i.argmax() for i in preds[0]]

In [96]:
print(classification_report(predictions_mod_2,test_labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22352
           1       1.00      1.00      1.00     23015

    accuracy                           1.00     45367
   macro avg       1.00      1.00      1.00     45367
weighted avg       1.00      1.00      1.00     45367



In [99]:
class IMDbDataset_test(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(test_labels)

In [100]:
test_nolab = IMDbDataset_test(test_encodings)

In [31]:
preds_mod_2_2 = trainer.predict(test_nolab)

In [84]:
print(classification_report([i.argmax() for i in preds_mod_2_2[0]],test_labels))

NameError: ignored

In [35]:
print(accuracy_score([i.argmax() for i in preds_mod_2_2[0]],test_labels))

0.9789274141997487


In [37]:
predictions = trainer.predict(test_dataset)

In [38]:
print(classification_report([i.argmax() for i in predictions[0]],predictions[1]))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     21435
           1       1.00      0.96      0.98     23932

    accuracy                           0.98     45367
   macro avg       0.98      0.98      0.98     45367
weighted avg       0.98      0.98      0.98     45367



In [40]:
predictions[2]

{'eval_loss': 0.09159401804208755,
 'eval_runtime': 341.1929,
 'eval_samples_per_second': 132.966}

In [31]:
real_test_df= pd.read_csv("/content/drive/MyDrive/real_test.csv",sep=";",header=None)

In [32]:
 real_test_df[0]

0     пичаально как же меня достал #сбер самовольным...
1     Уже несколько дней глюкавый  @sberbank глючит ...
2     В сбере онлайн можно поставить на заставки рус...
3     Так что, если есть хоть малейшая возможность н...
4     Сбер инвестор самое лютое говнище в которое я ...
5     Сбер увеличил почти в 1,5 раза доходы от комис...
6     Пришло оповещение от Сбербанка, что мол, средс...
7     'Который день не могу дочитать граф монте крис...
8                         ( пичаально. хочу в детство.'
9                   ( пичаально. хочу в детство опять.'
10              ( пичаально и грустно. хочу в детство.'
11    Там шаурмячная есть, чебуреки есть в ассортиме...
12    Блять, последнее совместное посещение чебуречн...
13    Не чебуречная Дружба на Сухаревской... Или у м...
14                      Тррр бум ба, Тррр ча ча. Да она
15    Там про сенрегацию аппартеида. Я охуел) И как ...
16    Ломающие новости: в Костроме группа синеботов ...
17              Это уже шестое покушение на скул

In [66]:
#real_texts = real_test_df[0].tolist()
real_texts_encodings = tokenizer(real_texts, truncation=True, padding=True)

In [65]:
real_texts[3] = "видимо фиолетовый лак отображает моё состояние:)"

In [35]:
class IMDbDataset_test(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(real_texts)

In [67]:
real_test = IMDbDataset_test(real_texts_encodings)

In [71]:
ev = trainer.evaluate()

In [73]:
ev

{'epoch': 1.0,
 'eval_loss': 0.00255760969594121,
 'eval_runtime': 146.108,
 'eval_samples_per_second': 248.405}

In [74]:
predictions = trainer.predict(real_test)

In [75]:
predictions[0][16]

array([ 3.4505424, -2.9496813], dtype=float32)

In [76]:
[i.argmax() for i in predictions[0]]

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [77]:
predictions[0][1]

array([ 3.456985 , -2.9567053], dtype=float32)

In [104]:
real_texts_encodings.items()

dict_items([('input_ids', [[101, 2739, 3629, 14198, 83227, 110, 10887, 1437, 81894, 2010, 74276, 866, 6030, 8382, 612, 44612, 128, 94595, 104189, 866, 869, 240, 130, 30561, 1469, 35109, 13776, 128, 4745, 27358, 1757, 23124, 8953, 128, 17611, 27466, 16988, 1469, 60309, 128, 5247, 1699, 14741, 5134, 128, 5318, 11690, 17652, 16972, 1469, 86899, 22443, 35014, 1465, 59385, 30377, 128, 2306, 8953, 3629, 128, 13322, 11471, 2226, 845, 17006, 57381, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 16578, 5325, 10333, 41939, 23983, 860, 168, 268, 12724, 21095, 84408, 7835, 12091, 13526, 132, 22395, 128, 1516, 130, 27324, 4725, 12313, 1699, 82722, 132, 19032, 79267, 880, 11654, 13002, 851, 87276, 852, 64872, 3075, 132, 777, 8133, 7036, 9098, 43752, 896, 130, 14179, 1699, 12620, 132, 110, 22403, 1437, 130, 2886, 12552, 132, 113271, 23983, 860, 132, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 781, 10887, 5839, 23449, 7021, 27942, 1469, 113586, 15184, 35269, 7465, 207, 11012, 12457, 13807, 

In [58]:
test_texts[:5]

['@link_oln :) то додам: бажаю тобі раю, твого особистого))',
 'зато фиолетовый лак теперь полностью отображает моё состояние:)',
 'Прохор Шаляпин принимает поздравления в связи с женитьбой на 57-летней миллионерше. Два любящих сердца наконец-то воссоединились :) Р...',
 'Который день не могу дочитать граф монте кристо((',
 'Теперь я начинаю понимать,все эти заумные цитаты!! ;( пичаально. хочу в детство.']

In [98]:
test_texts[1]

'зато фиолетовый лак теперь полностью отображает моё состояние:)'

In [99]:
test_texts[1] in train_texts

False

In [101]:
'@romanenkomax13 ну во все тяжкие это мура по сравнению с прослушкой) клан сопрано обожаю, но даже его она делает' in train_texts

True

In [102]:
intersection_set=set.intersection(set(train_texts),set(test_texts))

TF try

In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 7.7MB/s 
     |████████████████████████████████| 890kB 38.2MB/s 
     |████████████████████████████████| 3.2MB 50.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=642f71a35381e2996b5b55ca6fb8e51845244bceac587f011d9334cc8d08a447
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#load Pytorch model
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence" )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#save model to GoogleDrive
model.save_pretrained('/content/drive/MyDrive/rubert-base-cased-sentence/')

In [53]:
#create TF model from saved PyTorch model
from transformers import TFBertForSequenceClassification
#from transformers.configuration_bert import BertConfig
#num_labels = 2 
#config = BertConfig.from_pretrained("DeepPavlov/rubert-base-cased-sentence", num_labels=num_labels)
modelTF = TFBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/rubert-base-cased-sentence/", from_pt=True, num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [57]:
modelTF.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 177,854,978
Trainable params: 177,854,978
Non-trainable params: 0
_________________________________________________________________


In [13]:
#prepare DF
import pandas as pd
import numpy as np

In [19]:
#columns=['id','tdate','tmane','ttext','ttype','terp','trtw','tfav','tstcount','tfol','tfrien','listcount']

neg = pd.read_csv('https://raw.githubusercontent.com/tixonsit/mmdad_materials/master/negative.csv', encoding='utf8', sep=';', header=None)
pos = pd.read_csv('https://raw.githubusercontent.com/tixonsit/mmdad_materials/master/positive.csv', encoding='utf8', sep=';', header=None)
df = pos.append(neg, ignore_index = True)
df = df.sample(frac=1).reset_index(drop=True)
df[4] = df[4].map({-1:0, 1:1})
texts = df[3].values.tolist()
labels = df[4].values.tolist()

In [17]:
#train val test split
from sklearn.model_selection import train_test_split

In [20]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=2, stratify=labels, shuffle=True)
print("train_texts: ", len(train_texts))
print("test_texts: ", len(test_texts))
print("train_labels: ", len(train_labels))
print("test_labels: ",len(test_labels))

train_texts:  181467
test_texts:  45367
train_labels:  181467
test_labels:  45367


In [21]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=2, stratify=train_labels ,shuffle=True)
print("train_texts: ", len(train_texts))
print("val_texts: ", len(val_texts))
print("train_labels: ", len(train_labels))
print("val_labels: ", len(val_labels))

train_texts:  145173
val_texts:  36294
train_labels:  145173
val_labels:  36294


In [16]:
#tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [22]:
#tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [69]:
#prepare data with TF
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [60]:
#Fine-tuning with native TensorFlow
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
modelTF.compile(optimizer=optimizer, loss=loss,metrics=[metric]) # can also use any keras loss fn
#modelTF.fit(train_dataset.shuffle(1000).batch(16), epochs=1, batch_size=16)

In [103]:
modelTF.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 177,854,978
Trainable params: 177,854,978
Non-trainable params: 0
_________________________________________________________________


In [62]:
history = modelTF.fit(train_dataset.shuffle(1000).batch(16), epochs=1, batch_size=16)

9074/9074 [==============================] - 2516s 276ms/step - loss: 0.0594 - accuracy: 0.9770


In [65]:
modelTF.evaluate(val_dataset.batch(16))

2269/2269 [==============================] - 191s 83ms/step - loss: 0.0634 - accuracy: 0.9745


[0.06341072171926498, 0.9745412468910217]

In [108]:
modelTF.evaluate(test_dataset.batch(16))

2836/2836 [==============================] - 234s 82ms/step - loss: 0.0653 - accuracy: 0.9734


[0.06534941494464874, 0.9733947515487671]

In [68]:
y_pred = tf.nn.softmax(modelTF.predict(test_dataset))

ValueError: ignored

In [70]:
predictions = modelTF.predict(test_dataset)

In [73]:
y_pred = tf.nn.softmax(predictions[0])

In [74]:
y_pred_argmax = tf.math.argmax(y_pred, axis=1)

In [88]:
len(y_pred_argmax.numpy())

5897710

In [107]:
len(predictions[0])

5897710

In [92]:
test_dataset_2 = tf.data.Dataset.from_tensor_slices(dict(test_encodings))

In [93]:
predictions_2 = modelTF.predict(test_dataset_2)

In [102]:
predictions_2

TFSequenceClassifierOutput([('logits', array([[0.05306509, 0.6426067 ],
                                    [0.05268409, 0.6427597 ],
                                    [0.05261185, 0.6428069 ],
                                    ...,
                                    [0.05268758, 0.642753  ],
                                    [0.05268755, 0.6427529 ],
                                    [0.05268755, 0.6427529 ]], dtype=float32))])

In [98]:
len(test_labels)

45367

In [104]:
print(tokenizer.all_special_tokens)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [106]:
example = {'idx': tf.constant(1, dtype=tf.int64), 'label': tf.constant(0, dtype=tf.int64) ,
           'sentence': tf.constant('Все очень плохо и ужасно(', dtype=tf.string)}

In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_pred_argmax.numpy(),test_labels))

ValueError: ignored

In [33]:
predictions.logits

array([[0.44698602, 0.24500096],
       [0.44704372, 0.24496467],
       [0.44703448, 0.2449707 ],
       ...,
       [0.44703707, 0.24496901],
       [0.447037  , 0.24496903],
       [0.447037  , 0.24496903]], dtype=float32)

In [35]:
preds = tf.nn.softmax(predictions.logits, name=None)

In [48]:
preds

<tf.Tensor: shape=(5897710, 2), dtype=float32, numpy=
array([[0.5503253 , 0.4496747 ],
       [0.5503486 , 0.44965145],
       [0.55034477, 0.4496552 ],
       ...,
       [0.55034584, 0.4496542 ],
       [0.5503458 , 0.4496542 ],
       [0.5503458 , 0.4496542 ]], dtype=float32)>

In [43]:
len(test_labels)

45367

In [40]:
from sklearn.metrics import classification_report
print(classification_report(preds,test_labels))

ValueError: ignored